In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm, os
import tifffile

In [36]:
# Path to the folder containing the single channel Hoechst microscopy image used for TrackMate. It is important that
# nothing else is present in this folder.
date = '20210810'
scence = '1'

image_folder = r'D:/allen/data/mydata/ac_time_seperated/'+ date +'/merged/' + scence + '/'

# Path to the tracking csv file 'Spots in tracks statistics.csv' generated by TrackMate.
csv_p = image_folder + r"delgfp.csv"

# Path of the track curation folder which will contain the visualization of the tracks with a potential cell division.
track_curation_folder = image_folder + r'delgfp/'

# Track curation: for each track creates images of the detected cells in a folder, to be manually curated for cell division

filenames = os.listdir(image_folder)

saved_folder = r'D:/cycleclassification/vectorgfp/delgfp/' + date + '/' + scence + '/'

if not os.path.exists(saved_folder):
    os.makedirs(saved_folder)

In [37]:
filenames

['1.tif', 'All Spots statistics.csv', 'crop_gfp', 'delgfp.csv', 'gfptrack.csv']

In [38]:
img = tifffile.imread(image_folder + filenames[0])[:,1:,:,:]

In [39]:

# # Define the number of frames to use
n_frames = img.shape[0]

df = pd.read_csv(csv_p)

# Loop over all tracks and store the track id with potential division in 'split_track.npy'.
if not os.path.exists(track_curation_folder):
    os.mkdir(track_curation_folder)

In [40]:
if not os.path.exists(track_curation_folder + 'split_track.npy'):
    split_track = []
    for track in tqdm.tqdm(np.unique(df['TRACK_ID'])):
        if track != 'None':
#             print('track:',track)
            arr = df[df['TRACK_ID'] == track]['FRAME']
#             print('arr:',arr)
            frames, nID = np.unique(arr, return_counts=True)
#             print(nID)
            division_frame = np.where(nID >= 1)
            #print('division_frame:',division_frame)
            if len(division_frame[0]) > 0:
                if nID[division_frame[0][0]] == nID[-1]:
                    split_track = np.append(split_track, track)
    np.save(track_curation_folder + 'split_track.npy', split_track)
else:
    split_track = np.load(track_curation_folder + 'split_track.npy')

100%|██████████████████████████████████████████████████████████████████████████████| 285/285 [00:00<00:00, 2232.07it/s]


In [41]:
split_track

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [42]:
factor = 3.8496 # 1024/266
crop_size = 70 # half
image_width = 1024
for track_id in split_track:
    track_id = int(track_id)
    if not os.path.exists(saved_folder + '{}'.format(track_id)):
        os.makedirs(saved_folder + '{}'.format(track_id))

        # Get extreme coordinates of the track for image dimensions
        x_min, x_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor))
        y_min, y_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor))

        if np.min([x_min, x_max, y_min, y_max]) > crop_size and np.max([x_min, x_max, y_min, y_max]) < (1024-crop_size):
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
                print('im_bf:',im_bf.shape)
                tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

        else:
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                if x > crop_size and y < (image_width-crop_size) and x < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < 0.8*crop_size or x > (image_width-0.8*crop_size) or y < 0.8*crop_size or y > (image_width-0.8*crop_size):
                    pass  # get rid of cells too closing edge
                elif x > crop_size and x < (image_width-crop_size) and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width,x - crop_size:x + crop_size]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > crop_size and x < (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2,x - crop_size:x + crop_size]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, 0:crop_size*2]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, image_width -crop_size*2:image_width]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, 0:crop_size*2]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, 0:crop_size*2]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, image_width-crop_size*2:image_width]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                else:
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, image_width-crop_size*2:image_width]
#                     im_bf = np.concatenate((np.expand_dims(im_bf[3,:,:],axis=0),im_bf[0:3,:,:]),axis=0)
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]<ipython-input-42-1d4013db0083>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
<ipython-input-42-1d4013db0083>:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
  0%|                                                                                            | 0/9 [00:00<?, ?it/s]<ipython-input-42-1d4013db0083>:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
<ipython-input-42-1d4013db0083>:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y = np.int(df[df['TRACK_ID'

im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf:

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

 (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf:

  0%|                                                                                           | 0/51 [00:00<?, ?it/s]

...

im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf:

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

 (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 334.27it/s]

im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)


In [43]:
import gc

del img, im_bf
gc.collect()

758